<a href="https://colab.research.google.com/github/mmestres91/game_review_NLP_sentiment_analysis_/blob/main/review_sentiment_analysis_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
from sklearn.ensemble import RandomForestClassifier
import re
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler

from gensim.models import KeyedVectors
import gensim.downloader as api
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
import nltk

In [29]:
df = pd.read_csv('/content/drive/MyDrive/review_data.csv')
df.shape

(273656, 7)

In [30]:
df.head()

,Unnamed: 0,title,asin,overall,brand,reviewText,summary
0,0,Anno 2070,0700026657,5.0,Ubisoft,"This game is a bit hard to get the hang of, bu...",but when you do it's great.
1,1,Anno 2070,0700026657,4.0,Ubisoft,I played it a while but it was alright. The st...,"But in spite of that it was fun, I liked it"
2,2,Anno 2070,0700026657,3.0,Ubisoft,ok game.,Three Stars
3,3,Anno 2070,0700026657,2.0,Ubisoft,"found the game a bit too complicated, not what...",Two Stars
4,4,Anno 2070,0700026657,5.0,Ubisoft,"great game, I love it and have played it since...",love this game


In [31]:
#Data Preprocessing - Create Target Sentiment column, remove neutral reviews, set stop words and remove, parse out symbols, puncuations and numbers

df['sentiment'] = df['overall'].apply(lambda rating : +1 if rating > 3 else (-1 if rating == 3 else 0))

df = df.loc[df["sentiment"] >= 0]
df.reset_index(inplace=True)


nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stopwords = set(stop_words)
stopwords.update(["br", "href"])
df['reviewText'] = df['reviewText'].str.replace(r'[^\w\s]+', '')
df['reviewText'] = df['reviewText'].apply(lambda x: ' '.join([str(word) for word in str(x).split() if word not in (stopwords)]))
df['reviewText'] = df['reviewText'].str.replace('\d+', '')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
<ipython-input-31-b80f4c144ccd>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviewText'] = df['reviewText'].str.replace(r'[^\w\s]+', '')
<ipython-input-31-b80f4c144ccd>:13: FutureWarning: The default value of regex will change from True to False in a future version.
  df['reviewText'] = df['reviewText'].str.replace('\d+', '')


In [32]:
#Tokenize words in reviews

review_docs = [TaggedDocument(doc.split(' '), [i]) 
             for i, doc in enumerate(df.reviewText)]

In [33]:
#Initialize model and set parameters 

model = Doc2Vec(vector_size=64, window=2, min_count=1, workers=8, epochs = 15)

In [34]:
model.build_vocab(review_docs)
#train model
model.train(review_docs, total_examples=model.corpus_count
            , epochs=model.epochs)


In [35]:
#Generate vectors
card2vec = [model.infer_vector((df['reviewText'][i].split(' '))) 
  for i in range(0,len(df['reviewText']))]

In [36]:
#Calculate average of each word vector row
def Average(lst):
    return sum(lst) / len(lst)

avgList = []
for item in card2vec:
  average = Average(item)
  avgList.append(average)

In [37]:
#Set Avaerage Word Vector to Dataframe

df['review2vec'] = avgList
df_clean = df[['review2vec', 'sentiment']]

In [45]:
#Save Vectorized DataFrame

df_clean.to_csv('/content/drive/MyDrive/vec_data.csv')

In [46]:
#Load Vectorized DataFrame

df = pd.read_csv('/content/drive/MyDrive/vec_data.csv')

In [47]:
y = df.sentiment
X = df[['review2vec']]

In [48]:
print(X.shape)
print(y.shape)

(247914, 1)
(247914,)


In [49]:
# define oversampling strategy
oversample = RandomOverSampler(sampling_strategy='minority')
# fit and apply the transform
X_over, y_over = oversample.fit_resample(X, y)

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X_over, y_over, test_size = 0.2, random_state = 80)

In [43]:
model_rf = RandomForestClassifier(n_estimators = 10, criterion = 'entropy')
model_rf.fit(X_train, Y_train)

RandomForestClassifier(criterion='entropy', n_estimators=10)

In [44]:
# Predicting the Test set results
y_pred = model_rf.predict(X_test)
# Making the Confusion Matrix
print(classification_report(Y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.99      0.91     44263
           1       0.99      0.82      0.90     43847

    accuracy                           0.91     88110
   macro avg       0.92      0.90      0.90     88110
weighted avg       0.92      0.91      0.90     88110



In [ ]:
# Add in Confusion Matrix
# GridSearchCV to fine-tune parameters
# Explore Text Hero for more EDA